# Land cover map

### Resources
- https://geemap.org/notebooks/46_local_rf_training/

## Setup

In [1]:
from google.colab import drive
import pandas as pd
import geopandas as gpd
from shapely.geometry import mapping
import ee
import geemap
from geemap import ml

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/MyDrive/land_cover_classification_kaza

/content/drive/MyDrive/land_cover_classification_kaza


In [4]:
user_id = 'ee-alexvmt'
asset_name = 'mufunta_random_forest_trees'

In [5]:
ee.Authenticate()
ee.Initialize(project=user_id)

## Load imagery to be classified

In [6]:
# load roi
roi = gpd.read_file('data/Mufunta.shp')
roi

,WDPAID,WDPA_PID,PA_DEF,NAME,ORIG_NAME,DESIG,DESIG_ENG,DESIG_TYPE,IUCN_CAT,INT_CRIT,...,MANG_PLAN,VERIF,METADATAID,SUB_LOC,PARENT_ISO,ISO3,SUPP_INFO,CONS_OBJ,area_km2,geometry
0,555626090.0,555626090,1,Mufunta,Mufunta,Game Management Area,Game Management Area,National,Not Reported,Not Applicable,...,Not Reported,State Verified,943,Not Reported,ZMB,ZMB,Not Applicable,Not Applicable,6938.41,"POLYGON ((25.58421 -14.53576, 25.58513 -14.539..."


In [7]:
# turn roi polygon into list of points for clipping image
poly = roi[['geometry']].iloc[0][0]
poly_mapped = mapping(poly)
poly_coordinates = list(poly_mapped['coordinates'][0])

In [8]:
start_date = ee.Date('2023-01-01')
end_date = start_date.advance(365, 'day')
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']

In [9]:
image = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
  .filterDate(start_date, end_date)\
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))\
  .select(bands)\
  .median()\
  .clip(ee.Geometry.Polygon(poly_coordinates, proj=None))

## Create land cover map using classifier saved in the cloud

In [10]:
asset_id = 'projects/' + user_id + '/assets/' + asset_name

In [11]:
# read exported tree feature collection
rf_fc = ee.FeatureCollection(asset_id)

# convert it to a classifier, very similar to the `ml.trees_to_classifier` function
classifier = ml.fc_to_classifier(rf_fc)

# classify image
classified_image = image.classify(classifier)

In [12]:
center = [poly.centroid.y, poly.centroid.x]

In [13]:
# colors according to Dynamic World V1
palette = ['419bdf', # water
           'a59b8f', # bare
           'c4281b', # built up
           'e49635', # cropland
           '88b053', # grass
           'dfc35a', # shrub
           '397d49', # forest
           '7a87c6'] # wetland

In [14]:
legend_keys = ['Water', 'Bare', 'Built up', 'Cropland', 'Grass', 'Shrub', 'Forest', 'Wetland']

In [15]:
# display results
Map = geemap.Map(center=center, zoom=9)

Map.addLayer(
    image,
    {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000},
    'RGB image',
)

Map.addLayer(
    classified_image,
    {'min': 1, 'max': 8, 'palette': palette},
    'Land cover map',
)

Map.add_legend(keys=legend_keys, colors=palette, position='bottomleft')

Map

Map(center=[-15.33641924291382, 25.16615805807083], controls=(WidgetControl(options=['position', 'transparent_…

## Create land cover map using locally saved classifier

In [16]:
classifier = ml.csv_to_classifier('models/trees.csv')

In [17]:
classified_image = image.classify(classifier)

In [18]:
# display results
Map = geemap.Map(center=center, zoom=9)

Map.addLayer(
    image,
    {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000},
    'RGB image',
)

Map.addLayer(
    classified_image,
    {'min': 1, 'max': 8, 'palette': palette},
    'Land cover map',
)

Map.add_legend(keys=legend_keys, colors=palette, position='bottomleft')

Map

Map(center=[-15.33641924291382, 25.16615805807083], controls=(WidgetControl(options=['position', 'transparent_…

In [20]:
# save classified image
# upload to drive once task has finished
# use script classified_image.js in ee
task = ee.batch.Export.image.toDrive(
    image=classified_image,
    description='classified_image',
    folder='land_cover_classification_kaza',
    region=ee.Geometry.Polygon(poly_coordinates, proj=None),
    scale=10,
    maxPixels=1e13
)
task.start()